In [ ]:
import os

import corner
import astropy.units as u
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
import numpy as np
import h5py
import re

from thejoker.config import P_min, P_max
from thejoker.data import RVData
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import SimulatedRVOrbit

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
HIP102152_full_bounds = None

In [ ]:
# name = 'HIP11915_result'
# name = 'HIP102152_result'
# name = 'HIP102152_result_minus0'
# name = 'HIP102152_result_Pmin1'
name = 'HIP102152_result_jitter2'

short_name = "".join(re.search('(HIP[0-9]+)_result(.*)', name).groups())
print(short_name)

output_filename = "../cache/{}.h5".format(name)

In [ ]:
with h5py.File('../data/{}.h5'.format(name), 'r') as f:
    bmjd = f['mjd'][:]
    rv = quantity_from_hdf5(f, 'rv')
    rv_err = quantity_from_hdf5(f, 'rv_err')
data = RVData(bmjd, rv, stddev=rv_err)

In [ ]:
if HIP102152_full_bounds is None and name == 'HIP102152_result':
    HIP102152_full_bounds = (bmjd.min() - 50, bmjd.max() + 50)

In [ ]:
if '102152' in short_name:
    t_grid_min,t_grid_max = HIP102152_full_bounds
else:
    t_grid_min,t_grid_max = (bmjd.min() - 50, bmjd.max() + 50)

In [ ]:
with h5py.File(output_filename, 'r') as g:
    P = quantity_from_hdf5(g, 'P')
    asini = quantity_from_hdf5(g, 'asini')
    ecc = quantity_from_hdf5(g, 'ecc')
    omega = quantity_from_hdf5(g, 'omega')
    phi0 = quantity_from_hdf5(g, 'phi0')
    v0 = quantity_from_hdf5(g, 'v0')

In [ ]:
all_samples = np.vstack((np.log(P.to(u.day).value),
                         np.log(asini.to(u.R_sun).value),
                         ecc,
                         omega.to(u.degree).value % (360.),
                         phi0.to(u.degree).value,
                         v0.to(u.m/u.s).value)).T

labels = [r'$\ln (P/{\rm day})$', r'$\ln (a\,\sin i/R_\odot)$', '$e$', r'$\omega$ [deg]', 
          r'$\phi_0$ [deg]', '$v_0$ [m/s]']
ranges = [(-0.1, 9.1), (-10, 0), (0, 1), (0, 360), (0, 360), (-10, 10)]

In [ ]:
MAX_N_LINES = 128

# a time grid to plot RV curves of the model
t_grid = np.linspace(t_grid_min, t_grid_max, 1024)

# plot samples
fig,ax_rv = plt.subplots(figsize=(15,5))

print("{} good samples".format(len(P)))

# the number of lines to plot is at most 128, but may be fewer if we don't have
#   enough good samples
n_lines = min(len(P), MAX_N_LINES)

# scale the transparency of the lines
Q = 4. # HACK
line_alpha = 0.05 + Q / (n_lines + Q)

# plot orbits over the data
for i in range(len(P)):
    orbit = SimulatedRVOrbit(P=P[i], a_sin_i=asini[i], ecc=ecc[i],
                             omega=omega[i], phi0=phi0[i], v0=v0[[i]])
    model_rv = orbit.generate_rv_curve(t_grid - data.t_offset).to(u.m/u.s).value
    ax_rv.plot(t_grid, model_rv, linestyle='-', marker=None,
               alpha=line_alpha, color='#555555') #color='#3182bd')

    if i >= MAX_N_LINES:
        break

data.plot(ax=ax_rv, rv_unit=u.m/u.s, markersize=5)
ax_rv.set_xlim(t_grid.min(), t_grid.max())
_rv = data.rv.to(u.m/u.s).value
ax_rv.set_ylim(np.median(_rv)-10, np.median(_rv)+10)
ax_rv.set_xlabel('BMJD')
ax_rv.set_ylabel('RV [m s$^{-1}$]')

ax_rv.set_title(name)

fig.tight_layout()
fig.savefig("/Users/adrian/Downloads/{}-rv.png".format(short_name), dpi=300)

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(12,5))

axes[0].hist(np.log(P.value), bins=np.linspace(np.log(P.value).min()-1, np.log(P.value).max()+1, 32))
axes[1].hist(ecc, bins=np.linspace(0,1,32))

K = (2*np.pi*asini / (P * np.sqrt(1-ecc**2))).to(u.m/u.s).value
axes[2].hist(K, bins=np.linspace(K.min()-5,K.min()+5,16))

axes[0].set_xlabel('$P$ [day]')
axes[1].set_xlabel('$e$')
axes[2].set_xlabel('$K$ [m s$^{-1}$]')

fig.tight_layout()
fig.savefig("/Users/adrian/Downloads/{}-PeK.png".format(short_name), dpi=300)

In [ ]:
fig = corner.corner(all_samples, plot_contours=False, plot_density=False, plot_datapoints=True, 
                    data_kwargs=dict(alpha=1.), labels=labels, range=ranges)

fig.tight_layout()
fig.savefig("/Users/adrian/Downloads/{}-corner.png".format(short_name), dpi=300)

In [ ]:
all_samples_trans = np.vstack((np.log(P.to(u.day).value),
                               np.sqrt(asini.to(u.R_sun).value) * np.cos(phi0.to(u.radian).value),
                               np.sqrt(asini.to(u.R_sun).value) * np.sin(phi0.to(u.radian).value),
                               np.sqrt(ecc) * np.cos(omega).value,
                               np.sqrt(ecc) * np.sin(omega).value,
                               v0.to(u.m/u.s).value)).T

labels_trans = [r'$\ln P$', r'$\sqrt{a\,\sin i}\,\cos\phi_0$', r'$\sqrt{a\,\sin i}\,\sin\phi_0$',
                r'$\sqrt{e}\,\cos\omega$', r'$\sqrt{e}\,\sin\omega$', '$v_0$']

In [ ]:
fig = corner.corner(all_samples_trans, plot_contours=False, plot_density=False, plot_datapoints=True, 
                    data_kwargs=dict(alpha=1.), labels=labels_trans)

In [ ]:
with h5py.File(output_filename, 'r') as g:
    P = quantity_from_hdf5(g, 'P')
    asini = quantity_from_hdf5(g, 'asini')
    ecc = quantity_from_hdf5(g, 'ecc')
    omega = quantity_from_hdf5(g, 'omega')
    phi0 = quantity_from_hdf5(g, 'phi0')
    v0 = quantity_from_hdf5(g, 'v0')
    
    print(P)
    print(asini)

In [ ]:
asini / (2*np.pi / (P * np.sqrt(1-ecc**2))).value

In [ ]:
(2*np.pi*asini / (P * np.sqrt(1-ecc**2))).to(u.m/u.s)

In [ ]:
(2*np.pi*4.8*u.au / (3880*u.day * np.sqrt(1-0.1**2))).to(u.m/u.s)